In [1]:
# System imports
import sys
import os

# Add the parent directory of 'notebooks' to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move one level up
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Class/model imports
from data.data_loader import BillboardDataProcessor
from solver import Solver

# Package imports
import torch

# Select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is {device}")

Device is cuda


In [ ]:
# Download McGill Billboard data
!cd ../data && bash download_data.sh

In [2]:
# Generate the JAMs files from the billboard data
data_processer = BillboardDataProcessor(batch_size=64)
data_processer.process_billboard_data()

# Create data loeaders for train and test set
train_loader, test_loader, num_classes = data_processer.build_data_loaders()

print(f"Number of classes: {num_classes}")

Processed song 62 and appended data to combined CSV.
Processed song 3 and appended data to combined CSV.
Processed song 1096 and appended data to combined CSV.
Processed song 1009 and appended data to combined CSV.
Processed song 432 and appended data to combined CSV.
Processed song 730 and appended data to combined CSV.
Processed song 512 and appended data to combined CSV.
Processed song 979 and appended data to combined CSV.
Processed song 215 and appended data to combined CSV.
Processed song 1240 and appended data to combined CSV.
Processed song 1160 and appended data to combined CSV.
Processed song 762 and appended data to combined CSV.
Processed song 451 and appended data to combined CSV.
Processed song 555 and appended data to combined CSV.
Processed song 457 and appended data to combined CSV.
Processed song 393 and appended data to combined CSV.
Processed song 670 and appended data to combined CSV.
Processed song 1132 and appended data to combined CSV.
Processed song 731 and app

In [ ]:
# Initialize Solver for MLPChordClassifier
mlp_chord_classifier_solver = Solver(
    model_type="MLPChordClassifier",
    model_kwargs={"input_size": 24, "num_classes": num_classes},
    device=device,
    batch_size=128,
    learning_rate=0.001,
    epochs=20,
)

# Train and evaluate the model
mlp_chord_classifier_solver.train_and_evaluate(train_loader, test_loader, plot_results=True)

-----------------------------------
Epoch 1
-----------------------------------


Training:   0%|          | 0/51798 [00:00<?, ?it/s]

Training Loss: 2.7719. Validation Loss: 2.6489.
Training Accuracy: 0.4008. Validation Accuracy: 0.4125.
-----------------------------------
Epoch 2
-----------------------------------


Training:   0%|          | 0/51798 [00:00<?, ?it/s]

Training Loss: 2.6237. Validation Loss: 2.6102.
Training Accuracy: 0.4156. Validation Accuracy: 0.4179.
-----------------------------------
Epoch 3
-----------------------------------


Training:   0%|          | 0/51798 [00:00<?, ?it/s]

In [ ]:
# Initialize Solver for CRNNModel
crnn_model_solver = Solver(
    model_type="CRNNModel",
    model_kwargs={"input_features": 24, "num_classes": num_classes, "hidden_size": 128},
    device=device,
    batch_size=128,
    learning_rate=0.001,
    epochs=20,
)

crnn_model_solver.train_and_evaluate(train_loader, test_loader, plot_results=True)

In [ ]:
# Initialize Solver for CNNModel
cnn_model_solver = Solver(
    model_type="CNNModel",
    model_kwargs={"input_channels": 24, "num_classes": num_classes},
    device=device,
    batch_size=128,
    learning_rate=0.001,
    epochs=20,
)

cnn_model_solver.train_and_evaluate(train_loader, test_loader, plot_results=True)

In [ ]:
# Initialize Solver for RNNModel
rnn_model_solver = Solver(
    model_type="RNNModel",
    model_kwargs={"input_size": 24, "hidden_size": 128, "output_size": num_classes},
    device=device,
    batch_size=128,
    learning_rate=0.001,
    epochs=20,
)

rnn_model_solver.train_and_evaluate(train_loader, test_loader, plot_results=True)